# Building analysis with ETABS

## Connect to ETABS

In [4]:
import sys
import os
# Get the current working directory
current_folder = os.getcwd()
# Go up two directories
parent_folder = os.path.dirname(os.path.dirname(current_folder))
# Append the parent folder to the system path (for .ipynb files)
sys.path.append(parent_folder)
from etabsninja.interface import connect_to_etabs, test_etabs_connection
# Run this one time and that's it
SapModel,EtabsObject = connect_to_etabs()
test_etabs_connection()

ETABS connection test successful


## Elastic Analysis

In [9]:
import importlib
imported_module = importlib.import_module("etabsninja.get_functions")
importlib.reload(imported_module)
from etabsninja.get_functions import *
from etabsninja.settings import Settings
from icecream import ic # For better debugging

class Results:
    """Overall class to manage etabs model results."""

    def __init__(self):
        """Initialize the results."""
        self.settings = Settings()
        self.LoadCaseList = ["W Service"]
        self.building_height = 100 # ft
        self.JointsGroupName = "Nodos"
        self.max_BuildingDrift = 1/400
        self.max_InterstoryDrift = self.settings.max_InterstoryDrift

results = Results()
story_data = get_story_data(SapModel)
# frames = get_all_frames(SapModel)
# points = get_all_points(SapModel, inc_restraint=False)
LoadCaseList = ["W Service"]
building_height = 100 # ft
JointsGroupName = "Nodos"
max_BuildingDrift = 1/400
# story_drifts = get_StoryDriftsForStories(SapModel,LoadCaseList,max_InterstoryDrift)
# joint_drifts = get_StoryDriftsForJoints(SapModel,LoadCaseList,JointsGroupName,max_InterstoryDrift)
# CM_displacement = get_DiaphragmCMDisplacements(SapModel,LoadCaseList,building_height,max_BuildingDrift)

ic| max_InterstoryDrift: 0.0025
